In [14]:
# !pip install pythreejs
#!pip install numpy, scipy
# !pip install notebook==6.5.4
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

In [15]:
from pythreejs import * 
from IPython.display import display

import ipywidgets as widgets
import numpy as np
import time
from scipy.spatial.transform import Rotation as R
import asyncio

# Geometry

In [16]:
class Plane(Mesh):
    def __init__(self, origin=[0, 0, 0], normal=[0, 1, 0], size=10, color='gray', opacity=0.5):
        
        geometry = PlaneGeometry(width=size, height=size)
        
        material = MeshStandardMaterial(
            color=color, 
            transparent=True, 
            opacity=opacity, 
            side='DoubleSide')
        
        super().__init__(geometry=geometry, material=material)
        self.receiveShadow = True
        
        # Rotate the plane to the normal vector
        z = np.array([0, 0, 1])  # PlaneGeometry default normal
        axis = np.cross(z, normal)
        angle = np.arccos(np.dot(z, normal))  # Angle between vectors

        if np.linalg.norm(axis) > 1e-6:  # Avoid division by zero if vectors are parallel
            axis = axis / np.linalg.norm(axis)
            self.quaternion = list(np.append(axis * np.sin(angle / 2), np.cos(angle / 2)))  # Quaternion rotation 

        self.position = origin

class Cube(Mesh):
    def __init__(self, width=1.0, height=1.0, depth=1.0, color='green', position=[0, 0, 0], rotation=[0, 0, 0]):
        
        vertices, faces, edges, tets = self.generate_cube(width, height, depth)
        
        geometry = BufferGeometry(
            attributes={
                'position': BufferAttribute(array=vertices.copy(), normalized=False),
            },
            index=BufferAttribute(array=faces.copy().ravel(), normalized=False)
        )
        
        material = MeshStandardMaterial(
            color=color,  # RGB color of the material
            flatShading=True,
            wireframe=False,
        )
        super().__init__(geometry=geometry, material=material)
        
        self.castShadow = True

        # Transformation
        self.init_rotation = R.from_euler('XYZ', rotation, degrees=True)
        self.init_position = (np.array(position, dtype=np.float32) +  self.init_rotation.apply(self.geometry.attributes['position'].array)).copy()
        self.geometry.attributes['position'].array = self.init_position.copy()

        # Constraints Properties
        self.edges = edges.copy()
        self.rest_length = np.linalg.norm(self.init_position[edges[:, 0]].copy() - self.init_position[edges[:, 1]].copy(), axis=1)

        self.tets = tets.copy()
        self.rest_volume = np.einsum('ij,ij->i', np.cross(self.init_position[tets[:, 1]].copy() - self.init_position[tets[:, 0]].copy(), \
                                                          self.init_position[tets[:, 2]].copy() - self.init_position[tets[:, 0]].copy()
                                                          ), self.init_position[tets[:, 3]].copy() - self.init_position[tets[:, 0]].copy()) / 6.0
        swap_idx = np.where(self.rest_volume < 0)[0]
        self.tets[swap_idx[..., None], [1, 2]] = self.tets[swap_idx[..., None], [2, 1]]
        self.rest_volume = np.abs(self.rest_volume)
    
    def reset(self):
        self.geometry.attributes['position'].array = self.init_position.copy()
        
    
    def generate_cube(self, width=1.0, height=1.0, depth=1.0):
        """Generate vertices and faces for a box."""
        vertices = np.array([
            [-width/2, -height/2, -depth/2],  # 0 - Back Bottom Left
            [ width/2, -height/2, -depth/2],  # 1 - Back Bottom Right
            [ width/2,  height/2, -depth/2],  # 2 - Back Top Right
            [-width/2,  height/2, -depth/2],  # 3 - Back Top Left
            [-width/2, -height/2,  depth/2],  # 4 - Front Bottom Left
            [ width/2, -height/2,  depth/2],  # 5 - Front Bottom Right
            [ width/2,  height/2,  depth/2],  # 6 - Front Top Right
            [-width/2,  height/2,  depth/2],  # 7 - Front Top Left
            [0, 0, 0] # 8
        ], dtype=np.float32)

        # Define 12 triangles (6 faces x 2 triangles per face)
        faces = np.array([
            [4, 5, 6], [4, 6, 7],  # Front (+Z)
            [1, 0, 3], [1, 3, 2],  # Back (-Z)
            [5, 1, 2], [5, 2, 6],  # Right (+X)
            [0, 4, 7], [0, 7, 3],  # Left (-X)
            [3, 7, 6], [3, 6, 2],  # Top (+Y)
            [0, 1, 5], [0, 5, 4]   # Bottom (-Y)
        ], dtype=np.uint32)
        
        
        edges = np.array([
            [0, 1], [1, 2], [2, 3], [3, 0], # Back face
            [4, 5], [5, 6], [6, 7], [7, 4], # Front face
            [0, 4], [1, 5], [2, 6], [3, 7], # Connect front and back faces
            # [8, 0], [8, 1], [8, 2], [8, 3],  # Connect center to back face
            # [8, 4], [8, 5], [8, 6], [8, 7],   # Connect center to front face
            [0, 2], [1, 3], [4, 6], [5, 7], # Connect top and bottom faces
            [1, 6], [2, 5], [0, 7], [3, 4], # Connect front and back faces
            [0, 5], [1, 4], [2, 7], [3, 6], # Connect front and back faces
        ], dtype=np.uint32)
        
        
        tet_indices = np.array([
            [8, 0, 1, 2], [8, 0, 2, 3], # Back face
            [8, 4, 5, 6], [8, 4, 6, 7], # Front face
            [8, 0, 4, 5], [8, 0, 5, 1], # Connect front and back faces
            [8, 2, 6, 7], [8, 2, 7, 3], # Connect front
            [8, 5, 1, 6], [8, 1, 2, 6], # Connect front
            [8, 0, 4, 7], [8, 0, 7, 3], # Connect back
            
        ], dtype=np.uint32)
        
        return vertices, faces, edges, tet_indices

In [17]:
class DynamicObject:
    def __init__(self, mesh):
        self.mesh = mesh
        self.reset()
    
    def reset(self,):
        self.mesh.reset()
        
        self.currPos = self.mesh.init_position.copy()
        self.currVel = np.zeros_like(self.mesh.init_position)
        self.prevPos = self.mesh.init_position.copy()
        self.prevVel = np.zeros_like(self.mesh.init_position)

In [18]:
class Constraint:
    def solve(self, ):
        pass

class DistanceConstraint(Constraint):
    def __init__(self, body1, id1, body2, id2, rest_length, compliance=0.0, lambda_=0.0):
        # Body 1
        self.body1 = body1
        self.id1 = id1
        self.w1 = 1/self.body1.mass
        
        # Body 2
        self.body2 = body2
        self.id2 = id2
        self.w2 = 1/self.body2.mass
            
        # Constraint parameters
        self.rest_length = rest_length
        self.compliance = compliance
        self.lambda_ = lambda_
        
        
    def solve(self, h):
        # C(x1, x2) = ||x1 - x2|| - rest_length
        x1, x2 = self.body1.currPos[self.id1], self.body2.currPos[self.id2]
        
        n = x1 - x2
        d = np.linalg.norm(n)
        C = d - self.rest_length
        if d < 1e-6:
            return
        alpha = self.compliance / h / h
        
        dlambda = - (C + alpha * self.lambda_) / (self.w1 + self.w2 + alpha)
        self.lambda_ += dlambda        
        dx = dlambda * n / d

        self.body1.currPos[self.id1] += self.w1 * dx
        self.body2.currPos[self.id2] -= self.w2 * dx

class JointPositionConstraint(DistanceConstraint):
    def __init__(self, body1, id1, body2, id2, compliance=0.0, lambda_=0.0):
        super().__init__(body1, id1, body2, id2, rest_length=0, compliance=compliance, lambda_=lambda_)

    
class AttachmentConstraint(Constraint):
    def __init__(self, body, id, rest_position, compliance=0.0, lambda_=0.0):
        self.body = body
        self.id = id
        self.w = 1/self.body.mass
        
        self.rest_position = rest_position
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve(self, h):
        x = self.body.currPos[self.id]
        n = x - self.rest_position
        d = np.linalg.norm(n)
        
        # C(x) = ||x - x_rest||
        C = d
        if d < 1e-6:
            return
        
        alpha = self.compliance / h / h
        
        dlambda = - (C + alpha * self.lambda_) / (self.w + 0 + alpha)
        self.lambda_ += dlambda
        dx = dlambda * n / d
        
        self.body.currPos[self.id] += self.w * dx
        

    
class PlaneCollisionConstraint(Constraint):
    def __init__(self, body, id, plane, q, normal, depth, compliance=0.0, lambda_=0.0):
        self.body = body
        self.id = id # contact id
        self.w = 1/self.body.mass
        
        self.plane = plane
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve_position(self, h):
        # Plane : n • x + d = 0
        n = self.plane.normal
        d = -np.dot(n, self.plane.offset)
                
        x_prev = self.body.prevPos[self.id]
        x = self.body.currPos[self.id]
        ray = x - x_prev
        ray_start = np.dot(n, x_prev) + d
        ray_proj = np.dot(n, ray)
        
        t = -ray_start / ray_proj
        
        # Continuous Collision
        if 0 < t and t <= 1:
            # Ray Intersection with the plane
            q_c = x_prev + t * ray
            
            # C(x) = (x - q_c) • n = 0
            C = np.dot(x - q_c, n)
            dC = n
 
            # Inequality Constraint
            if C >= 0:
                return
                    
        # Static Collision
        if t <= 0:
            # Projection of x on the plane
            q_s = x - np.dot(x - self.plane.offset, n) * n
            
            # C(x) = (x - q_s) • n = 0
            C = np.dot(x - q_s, n)
            dC = n
            
            # Inequality Constraint
            if C >= 0:
                return
            
        # Compliance
        alpha = self.compliance / h / h
        
        dlambda = - (C + alpha * self.lambda_) / (self.w * np.dot(dC, dC) + alpha)
        self.lambda_ += dlambda
        dx = dlambda * dC
        
        self.body.currPos[self.id] += self.w * dx

            
            
    def solve_velocity(self, h):
        v = self.body.currVel[self.id]
        n = self.plane.normal
        
        v_n = np.dot(v, n) * n
        v_t = v - v_n
        
        self.body.currVel[self.id] = - v_n * self.body.restitution + v_t * self.body.friction
        
    
class DynamicCollisionConstraint(Constraint):
    def __init__(self, body1, x1, body2, x2, compliance=0.0, lambda_=0.0):
        self.body1 = body1
        self.x1 = x1
        
        self.body2 = body2
        self.x2 = x2
        
        self.compliance = compliance
        self.lambda_ = lambda_
        
    def solve_position(self, ):
        pass
    
    def solve_velocity(self, ):
        pass

In [19]:
class PBDSimulation:
    def __init__(self, time_step=0.0333, substeps=1, gravity=[0, -9.8, 0]):
        
        self.bodies = []
        self.constraints = []
        
        self.gravity = gravity
        self.time_step = time_step
        self.substeps = substeps
        self.h = self.time_step/self.substeps
        
        
    def add_constraint(self, constraint):
        self.constraints.append(constraint)
    
    def add_body(self, body):
        if isinstance(body, list):
            self.bodies.extend(body)
        else:
            self.bodies.append(body)
        
    def step(self,):        
        
        # Gather collision candidates
        collisions = self.check_collisions()

        for _ in range(self.substeps):
            # Check actual collisions
            contacts = self.check_contacts(collisions)
            
            # Integrate
            self.integrate()
            
            # Solve constraints
            self.solve_constraints(contacts)
            
            # Update velocities
            self.update_velocities()            
            
            # Solve velocities
            self.solve_velocities(contacts)
         

    def check_collisions(self,):
        # AABB or BVH
        # Broad Phase
        
        # Body and Plane
        for body in self.bodies:
            
            pass
    
    
    def check_contacts(self, collisions):
        contacts = []
        
        for (body1, body2) in collisions:
            n, d, p1, p2 = self.compute_collision(body1, body2)
            if d > 0:
                if isinstance(body1, DynamicObject) and isinstance(body2, DynamicObject):
                    contacts.append(DynamicCollisionConstraint(body1, p1, body2, p2))
    
                elif isinstance(body1, DynamicObject):
                    contacts.append(PlaneCollisionConstraint(body1, p1, body2, p2, n, d))
    
                elif isinstance(body2, DynamicObject):
                    contacts.append(PlaneCollisionConstraint(body2, p2, body1, p1, n, d))
                    
        return contacts
    
    
    def integrate(self,):
        for body in self.bodies:
            body.prevPos = body.currPos.copy()
            body.currVel += self.gravity * self.h
            body.currPos += body.currVel * self.h
            
    
    def solve_constraints(self, contacts):
        for constraint in self.constraints:
            constraint.solve(self.h)
            
        for contact in contacts:
            contact.solve_position(self.h)
        
    
    def update_velocities(self,):
        for body in self.bodies:
            body.currVel = (body.currPos - body.prevPos) / self.h
    
    
    def solve_velocities(self, contacts):
        for contact in contacts:
            contact.solve_velocity()    
    
    def reset(self,):
        for body in self.bodies:
            body.reset()
            
        for constraint in self.constraints:
            constraint.lambda_ = 0.0
        

In [20]:
if __name__ == '__main__':

    ## Place Objects
    cube1 = Cube(width=1.0, height=12, depth=1, color='yellow', position=[0, 4.0, 0])
    cube2 = Cube(width=1, height=2, depth=1, color='blue', position=[0, 2, 0])
    cube3 = Cube(width=1, height=1, depth=1, color='red', position=[1.0, 0.5, 0], rotation=[0, 0, 90])
    
    plane = Plane(origin=[0, 0, 0], normal=[0, 1, 0], size=10000, color='gray', opacity=1.0)
    
    pbd = PBDSimulation()
    pbd.add_body([cube1, cube2, cube3])
    
    
    ## Generate Constraint
    pbd.add_constraint(DistanceConstraint)




    ## Render
    width, height = 800, 800
    scene = Scene(children=[cube1, cube2, cube3, plane], background='skyblue')
    camera = PerspectiveCamera(position=[0, 5, 10], aspect=width/height)
    renderer = Renderer(scene=scene, camera=camera, width=width, height=height)
    
    renderer.controls.append(OrbitControls(controlling=renderer.camera))


    ## Widgets
    running = False
    
    async def animate():
        global running
        while running:
            pbd.step()
            await asyncio.sleep(0.0333)
    
    def play_animation(change):
        global running
        running = not running
        if running:
            asyncio.create_task(animate())
    
    def reset(change):
        global running
        running = False
        cube1.reset()
        cube2.reset()
        cube3.reset()
    
    
    play_button = widgets.Button(description='START/STOP', button_style='success')
    play_button.on_click(play_animation)
    
    reset_button = widgets.Button(description='RESET', button_style='danger')
    reset_button.on_click(reset)
    
    buttons = widgets.HBox([play_button, reset_button])
    display(widgets.VBox([renderer, buttons]))
    



In [21]:
def solve_volume_constraint(x1, x2, x3, x4, w1, w2, w3, w4, rest_volume, compliance, h, prev_lambda=None):
    """
    Solves the volume conservation constraint for a tetrahedron.
    """
    # Compute current volume using the triple scalar product
    volume = np.dot(np.cross(x2 - x1, x3 - x1), (x4 - x1)) / 6.0

    C = volume - rest_volume  # Volume constraint violation
    
    # If the volume is nearly zero, avoid instability
    if abs(C) < 1e-6:
        return np.zeros_like(x1), np.zeros_like(x2), np.zeros_like(x3), np.zeros_like(x4), 0

    # Compute the gradients for all four vertices
    grad_x1 = -np.cross(x4 - x2, x3 - x1) / 6.0
    grad_x2 = -np.cross(x3 - x1, x4 - x2) / 6.0
    grad_x3 = -np.cross(x4 - x1, x2 - x1) / 6.0
    grad_x4 = -np.cross(x2 - x1, x3 - x1) / 6.0

    # Compute the sum of weighted gradients
    weighted_sum = w1 * np.dot(grad_x1, grad_x1) + w2 * np.dot(grad_x2, grad_x2) \
                 + w3 * np.dot(grad_x3, grad_x3) + w4 * np.dot(grad_x4, grad_x4)

    # Compliance term (XPBD-style)
    alpha = compliance / (h * h)
    
    # Solve for Lagrange multiplier update
    dlambda = - (C + alpha * prev_lambda) / (weighted_sum + alpha)
    new_lambda = prev_lambda + dlambda

    # Compute displacement updates for each vertex
    dx1 = -w1 * dlambda * grad_x1
    dx2 = -w2 * dlambda * grad_x2
    dx3 = -w3 * dlambda * grad_x3
    dx4 = -w4 * dlambda * grad_x4

    return dx1, dx2, dx3, dx4, new_lambda

def solve_distance_constraint(x1, x2, w1, w2, rest_length, compliance, h, prev_lambda=None):
    n = (x1 - x2)
    d = np.linalg.norm(n)
    # C(x1, x2) = |x1 - x2| - rest_length
    C = d - rest_length 
    
    if d < 1e-6:
        return 0, 0, 0
    n /= d
    
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = -w2 * dx
    
    return dx1, dx2, new_lambda

def static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=None):

    alpha = compliance / (h*h)
    
    # Check vertex-object intersection    
    d = -np.dot(normal, offset)
    ray = x1 - x1_prev
    ray_start = np.dot(x1_prev, normal) + d
    ray_projection = np.dot(ray, normal)
    
    # if abs(ray_projection) < 1e-13:
    #     return 0, None, None
    
    t = - ray_start / ray_projection
    

    # Continuous Collision
    if t > 0 and t <= 1:
        q_c = x1_prev + t * ray
    
        # C(x1) = (x1 - q) * n
        C = (x1 - q_c) * normal
        if np.all(C >= 0): # inequality constraint
            return 0, None, None
        dC = normal
        dx = -C * dC / (np.dot(dC, dC) + alpha)
        

        return dx, None, None
    
    # Static collision
    if t <= 0:
        q_s = x1 - (x1 - offset) * normal
        C = (x1 - q_s) * normal
        if np.all(C >= 0):
            return 0, None, None
   
        dC = normal
        dx = -C * dC / (np.dot(dC, dC) + alpha)     

        return dx, None, None

    # No Collision 
    if t > 1:
        return 0, None, None

import asyncio
async def step(bodies):
    
    gravity = np.array([0, -9.81, 0])
    time_step = 1/30
    substeps = 5
    h = time_step/substeps
    
    # for body in bodies:
    #     body.currPos = body.geometry.attributes['position'].array.copy()
    #     body.currVel = np.zeros(body.currPos.shape)
    # while True:
        ## Gather Collisions Candidates
    coll_idx = []
    for body in bodies:
        coll_idx.append(np.where(body.currPos[:, 1]<0)[0])
        
    for _ in range(substeps):
        ## Check Actual Collisions
        
        ## integration
        for body in bodies:
            body.prevPos = body.currPos.copy()
            body.currVel += gravity * h * 1.0
            body.currPos += body.currVel * h
            
            
        # Solve Distance Constraints
        for body in bodies:
            for (i, j), rest_length in zip(body.edges, body.rest_length):
                x1, x2 = body.currPos[i], body.currPos[j]
                dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=rest_length, compliance=0.000010, h=h, prev_lambda=0)
                body.currPos[i] += dx1
                body.currPos[j] += dx2
    

        # Solve Volume Constraints
        # for body in bodies:
        #     for (i, j, k, l), rest_volume in zip(body.tets, body.rest_volume):
        #         x1, x2, x3, x4 = body.currPos[i], body.currPos[j], body.currPos[k], body.currPos[l]
        #         dx1, dx2, dx3, dx4, _ = solve_volume_constraint(x1, x2, x3, x4, 1, 1, 1, 1, rest_volume=rest_volume, compliance=10.0, h=h, prev_lambda=0)
        #         body.currPos[i] += dx1
        #         body.currPos[j] += dx2
        #         body.currPos[k] += dx3
        #         body.currPos[l] += dx4

        ## Solve Joint Constraints
        # 1 cube1 (5, 1) - cube2 (6, 2)
        # x1, x2 = cube1.currPos[5], cube2.currPos[6]
        # dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        # cube1.currPos[5] += dx1
        # cube2.currPos[6] += dx2
        
        # x1, x2 = cube1.currPos[1], cube2.currPos[2]
        # dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        # cube1.currPos[1] += dx1
        # cube2.currPos[2] += dx2
        
        # x1, x2 = cube2.currPos[4], cube3.currPos[7]
        # dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        # cube2.currPos[4] += dx1
        # cube3.currPos[7] += dx2
        
        # x1, x2 = cube2.currPos[0], cube3.currPos[3]
        # dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        # cube2.currPos[0] += dx1
        # cube3.currPos[3] += dx2
        
        
        
        ## Solve Collisons
        for n in range(len(bodies)):
            for i in coll_idx[n]:
                x1 = bodies[n].currPos[i]
                x1_prev = bodies[n].prevPos[i]
                normal = np.array([0.0, 1.0, 0.0])
                offset = np.array([0.0, 0.0, 0.0])

                dx, _, _ = static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0.0000001, prev_lambda=0)

                bodies[n].currPos[i] += dx

        
        ## Updata Velocities with solved positions
        for body in bodies: 
            body.currVel = (body.currPos - body.prevPos) / h            
    
    
            
        ## Solve Velocities  - Damping and Friction
        for n in range(len(bodies)):

            for i in coll_idx[n]:
                v_n = np.dot(bodies[n].currVel[i], normal) * normal
                v_t = bodies[n].currVel[i] - v_n
                bodies[n].currVel[i] = -v_n * 0.3 + v_t * 0.6

    
    ## Update the geometry
    # print(bodies[1].currPos[:, 1])
    for body in bodies:
        body.geometry.attributes['position'].array = body.currPos.copy()
    
    await asyncio.sleep(0.033)
    # time.sleep(0.033)

In [22]:
# Rendering setup
width, height = 800, 600 
aspect = width / height

scene = Scene(background='black')  # Background color of the scene
camera = PerspectiveCamera(
    position=[5, 5, 10], 
    lookAt=[0, 0, 0],
    aspect=aspect
    )

ambient_light = AmbientLight(color='white', intensity=1.0)  # Soft overall light
directional_light = DirectionalLight(
    color='white',
    intensity=2.0,        # Brightness
    position=[0, 50, 100],   # Light direction
    
)
directional_light.castShadow = True  # Enable shadow casting

spot_light = SpotLight(
    color='white',
    intensity=10.0,        # Increase brightness
    position=[0, 10, 0],   # Place the light above the cube
    distance=10,          # Light range (covers cube)
    angle=0.5,            # Cone angle (medium focus)
    penumbra=0.3,         # Soft shadow edges
    decay=2,              # Realistic falloff
)
spot_light.castShadow = True  # Enable shadow casting
scene.add([ambient_light, directional_light])



controller = OrbitControls(controlling=camera,)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.shadowMap.enabled = True
renderer.shadowMap.type = 'PCFSoftShadowMap'


# Scene setup
cube1 = Cube(width=2.0, height=2.0, depth=2.0, color='yellow', position=[-1.0, 1.0, 0])
cube2 = Cube(width=2.0, height=2.0, depth=2.0, color='blue', position=[1.0, 1.0, 0])
cube3 = Cube(width=2.0, height=2.0, depth=2.0, color='red', position=[-1.0, 3.0, 0],)
cube4 = Cube(width=2.0, height=2.0, depth=2.0, color='green', position=[1.0, 3.0, 0],)


scene.add(cube1)
scene.add(cube2)
scene.add(cube3)
scene.add(cube4)
bodies = [cube1, cube2, cube3, cube4]

plane = Plane(normal=[0, 1, 0], size=1000, color='gray', opacity=1.0)
scene.add([plane])

## Animation State

running = False  # Global flag

for body in bodies:
    body.currPos = body.geometry.attributes['position'].array.copy()
    body.currVel = np.zeros(body.currPos.shape)
    
async def animate():
    global running    
    while running:
        await step(bodies)

def toggle_animation(_):
    global running
    if running:
        running = False
    else:
        running = True
        asyncio.create_task(animate())

def reset(_):
    global running
    running = False
    
    for body in bodies:
        body.currPos = body.init_position.copy()
        body.currVel = np.zeros(body.currPos.shape)
        body.prevPos = body.currPos.copy()
        body.prevVel = body.currVel.copy()
        body.geometry.attributes['position'].array = body.currPos.copy()


start_stop_button = widgets.Button(description="START / STOP", button_style='success')
start_stop_button.on_click(toggle_animation)

reset_button = widgets.Button(description="RESET", button_style='danger')
reset_button.on_click(reset)



# Display widgets
buttons =  widgets.HBox([reset_button, start_stop_button])
display(widgets.VBox([renderer, buttons]))


In [23]:
def solve_volume_constraint(x1, x2, x3, x4, w1, w2, w3, w4, rest_volume, compliance, h, prev_lambda=None):
    """
    Solves the volume conservation constraint for a tetrahedron.
    """
    # Compute current volume using the triple scalar product
    volume = np.dot(np.cross(x2 - x1, x3 - x1), (x4 - x1)) / 6.0

    C = volume - rest_volume  # Volume constraint violation
    
    # If the volume is nearly zero, avoid instability
    if abs(C) < 1e-6:
        return np.zeros_like(x1), np.zeros_like(x2), np.zeros_like(x3), np.zeros_like(x4), 0

    # Compute the gradients for all four vertices
    grad_x1 = -np.cross(x4 - x2, x3 - x1) / 6.0
    grad_x2 = -np.cross(x3 - x1, x4 - x2) / 6.0
    grad_x3 = -np.cross(x4 - x1, x2 - x1) / 6.0
    grad_x4 = -np.cross(x2 - x1, x3 - x1) / 6.0

    # Compute the sum of weighted gradients
    weighted_sum = w1 * np.dot(grad_x1, grad_x1) + w2 * np.dot(grad_x2, grad_x2) \
                 + w3 * np.dot(grad_x3, grad_x3) + w4 * np.dot(grad_x4, grad_x4)

    # Compliance term (XPBD-style)
    alpha = compliance / (h * h)
    
    # Solve for Lagrange multiplier update
    dlambda = - (C + alpha * prev_lambda) / (weighted_sum + alpha)
    new_lambda = prev_lambda + dlambda

    # Compute displacement updates for each vertex
    dx1 = -w1 * dlambda * grad_x1
    dx2 = -w2 * dlambda * grad_x2
    dx3 = -w3 * dlambda * grad_x3
    dx4 = -w4 * dlambda * grad_x4

    return dx1, dx2, dx3, dx4, new_lambda

def solve_distance_constraint(x1, x2, w1, w2, rest_length, compliance, h, prev_lambda=None):
    n = (x1 - x2)
    d = np.linalg.norm(n)
    # C(x1, x2) = |x1 - x2| - rest_length
    C = d - rest_length 
    
    if d < 1e-6:
        return 0, 0, 0
    n /= d
    
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = -w2 * dx
    
    return dx1, dx2, new_lambda

def static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=None):

    alpha = compliance / (h*h)
    
    # Check vertex-object intersection    
    d = -np.dot(normal, offset)
    ray = x1 - x1_prev
    ray_start = np.dot(x1_prev, normal) + d
    ray_projection = np.dot(ray, normal)
    
    # if abs(ray_projection) < 1e-13:
    #     return 0, None, None
    
    t = - ray_start / ray_projection
    

    # Continuous Collision
    if t > 0 and t <= 1:
        q_c = x1_prev + t * ray
    
        # C(x1) = (x1 - q) * n
        C = (x1 - q_c) * normal
        if np.all(C >= 0): # inequality constraint
            return 0, None, None
        dC = normal
        dx = -C * dC / (np.dot(dC, dC) + alpha)
        

        return dx, None, None
    
    # Static collision
    if t <= 0:
        q_s = x1 - (x1 - offset) * normal
        C = (x1 - q_s) * normal
        if np.all(C >= 0):
            return 0, None, None
   
        dC = normal
        dx = -C * dC / (np.dot(dC, dC) + alpha)     

        return dx, None, None

    # No Collision 
    if t > 1:
        return 0, None, None

# Simulation parameters

gravity = np.array([0, -9.81, 0])
time_step = 1/30
substeps = 5
h = time_step/substeps

for body in bodies:
    body.currPos = body.geometry.attributes['position'].array.copy()
    body.currVel = np.zeros(body.currPos.shape)
while True:
    ## Gather Collisions Candidates
    coll_idx = []
    for body in bodies:
        coll_idx.append(np.where(body.currPos[:, 1]<0)[0])
    for _ in range(substeps):
        ## Check Actual Collisions
        
        ## integration
        for body in bodies:
            body.prevPos = body.currPos.copy()
            body.currVel += gravity * h * 1.0
            body.currPos += body.currVel * h
            
            
        # Solve Distance Constraints
        for body in bodies:
            for (i, j), rest_length in zip(body.edges, body.rest_length):
                x1, x2 = body.currPos[i], body.currPos[j]
                dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=rest_length, compliance=0.000010, h=h, prev_lambda=0)
                body.currPos[i] += dx1
                body.currPos[j] += dx2
    

        # Solve Volume Constraints
        # for body in bodies:
        #     for (i, j, k, l), rest_volume in zip(body.tets, body.rest_volume):
        #         x1, x2, x3, x4 = body.currPos[i], body.currPos[j], body.currPos[k], body.currPos[l]
        #         dx1, dx2, dx3, dx4, _ = solve_volume_constraint(x1, x2, x3, x4, 1, 1, 1, 1, rest_volume=rest_volume, compliance=10.0, h=h, prev_lambda=0)
        #         body.currPos[i] += dx1
        #         body.currPos[j] += dx2
        #         body.currPos[k] += dx3
        #         body.currPos[l] += dx4

        ## Solve Joint Constraints
        # 1 cube1 (5, 1) - cube2 (6, 2)
        x1, x2 = cube1.currPos[5], cube2.currPos[6]
        dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        cube1.currPos[5] += dx1
        cube2.currPos[6] += dx2
        
        x1, x2 = cube1.currPos[1], cube2.currPos[2]
        dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        cube1.currPos[1] += dx1
        cube2.currPos[2] += dx2
        
        x1, x2 = cube2.currPos[5], cube3.currPos[7]
        dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        cube2.currPos[4] += dx1
        cube3.currPos[7] += dx2
        
        x1, x2 = cube2.currPos[1], cube3.currPos[3]
        dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=0, compliance=0.0, h=h, prev_lambda=0)
        cube2.currPos[0] += dx1
        cube3.currPos[3] += dx2
        
        
        
        ## Solve Collisons
        for n in range(len(bodies)):
            for i in coll_idx[n]:
                x1 = bodies[n].currPos[i]
                x1_prev = bodies[n].prevPos[i]
                normal = np.array([0.0, 1.0, 0.0])
                offset = np.array([0.0, 0.0, 0.0])

                dx, _, _ = static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0.0000001, prev_lambda=0)

                bodies[n].currPos[i] += dx

        
        ## Updata Velocities with solved positions
        for body in bodies: 
            body.currVel = (body.currPos - body.prevPos) / h            
    
    
            
        ## Solve Velocities  - Damping and Friction
        for n in range(len(bodies)):

            for i in coll_idx[n]:
                v_n = np.dot(bodies[n].currVel[i], normal) * normal
                v_t = bodies[n].currVel[i] - v_n
                bodies[n].currVel[i] = -v_n * 0.3 + v_t * 0.6

    
    ## Update the geometry
    # print(bodies[1].currPos[:, 1])
    for body in bodies:
        body.geometry.attributes['position'].array = body.currPos.copy()
    
    
    time.sleep(0.033)

KeyboardInterrupt: 